## Reproduction Analysis with Self-designed Python Package  

written by **Jinwoo Lee**.        
jil527@ucsd.edu | jinwoo-lee.com

Nov 6th, 2025  
as a PSYC201A's Final Project

Using the two preprocessed datasets, `df-behav-young_pped.csv` and `df-tract-young_pped.csv`, generated through `01_data-sanity-check.ipynb`, we will conduct the Anna Karenina Testing and Mantel Permutation Test.   

For this purpose, we will utilize parts of the **‘DIM’** package, which was custom-developed for the First Year Project.
Since **DIM** is part of an ongoing research project in our lab, the corresponding GitHub repository is not publicly available.

In [1]:
import sys
import torch
import numpy as np
import pandas as pd

sys.path.insert(0, '../DIM/src')
from lib.dim.level1 import Level1
from lib.utils.distances import weighted_euclidean_distance

In [2]:
df_behav = pd.read_csv('../data-from-authors/df-behav-young_pped.csv')
df_tract = pd.read_csv('../data-from-authors/df-tract-young_pped.csv')

In [3]:
print(df_behav.shape)
print(df_tract.shape)

(119, 2)
(119, 14770)


In [4]:
## making a Euclidean-distance-based dissimilarity matrix for tractography data
df_tract_torch = torch.Tensor(df_tract.values)
equal_weights_torch = torch.ones(df_tract_torch.shape[1], dtype = torch.float32)

D_tract_torch = weighted_euclidean_distance(df_tract_torch, equal_weights_torch)
D_tract = D_tract_torch.numpy()

In [5]:
equal_weights_torch

tensor([1., 1., 1.,  ..., 1., 1., 1.])

In [6]:
stai = df_behav[['STAI_Trait_Anxiety']].to_numpy()

In [12]:
## model fitting
AnnaK_model = Level1(
    model = "AnnaK",
    distance = "Mean",
    weighting = "None",
    dependency = "spearman_r",
    normalize = True
)

Validation successful!


In [13]:
AnnaK_model.fit(stai, D_tract)

In [14]:
mantel_result = AnnaK_model.permutation_test(
    n_perms = 10000,
    alternative = "two-sided",
    seed = 42,
    verbose = True,
    return_null_dist = True
)

Permutation test: 100%|██████████| 10000/10000 [00:13<00:00, 753.86perm/s]


In [15]:
mantel_result

{'observed_stat': -0.01765722967684269,
 'p_value': np.float64(0.8382),
 'n_perms': 10000,
 'alternative': 'two-sided',
 'null_mean': np.float64(-0.0006863195481098956),
 'null_std': np.float64(0.08414081483745552),
 'null_distribution': array([-0.0620001 ,  0.24066629,  0.02601696, ...,  0.02965535,
         0.12057251,  0.10045934], shape=(10000,))}